In [1]:
from ray.rllib.agents.ppo import PPOTrainer

In [2]:
import gym, ray
import numpy as np

Note:

- This is a two-player game. So, maybe we could fix one player to always do the same thing, and then learn against that?
- Or, we could learn two agents?
- Or ???

In [3]:
import copy
import requests

In [19]:
class IteratedPDEnv(gym.Env):
    
    INITIAL_STATE = {"totalPlayer2": 0, "totalPlayer1": 0}
    
    state = None
    
    def __init__ (self, env_config):
        # Note: Our action space is for ONE player; namely, exactly two
        # choices:
        #  - Defect
        #  - Cooperate
        self.action_space      = gym.spaces.Discrete(2)
        
        # Our observation is simply the total jail time for ourselves (Player 1)
        self.observation_space = gym.spaces.Box(
                                      low=0
                                    , high=np.iinfo(np.int32).max
                                    , shape=(1,)
                                    , dtype=np.int32
                                    )
        # self.seed(1)
        self.reset()
    
    
    def reset (self):
        self.state = copy.deepcopy(self.INITIAL_STATE)
    
    
    def step (self, action):
        # action is either 0 or 1.
        
        if action == 0:
            player1Action = "Cooperate"
        else:
            player1Action = "Defect"
        
        assert action in [0, 1], "Unknown action!"
        
        data = { "player1Action": player1Action
               , "player2Action": "Cooperate" # Goody-two-shoes
               , "startingState": self.state
               }
        
        # Do a post to the server; get the payoffs.
        response = requests.post("http://localhost:3000/play", json=data).json()
        
        print(response)
        
        # Update the state.
        self.state["totalPlayer1"] += response["lastPlayer1Payoff"]
        self.state["totalPlayer2"] += response["lastPlayer2Payoff"]
        
        return [self.state, self.state["totalPlayer1"], False, {}]

In [25]:
env = IteratedPDEnv(env_config = {})

In [26]:
env.step(action=1)

{'lastPlayer1Payoff': 5, 'lastPlayer2Payoff': 0, 'gameState': {'totalPlayer2': 0, 'totalPlayer1': 5}}


[{'totalPlayer2': 0, 'totalPlayer1': 5}, 5, False, {}]

### Let's try training it!

In [7]:
# ray.init()

{'node_ip_address': '192.168.1.212',
 'raylet_ip_address': '192.168.1.212',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-30_10-47-56_980375_3715266/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-30_10-47-56_980375_3715266/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-30_10-47-56_980375_3715266',
 'metrics_export_port': 65133,
 'gcs_address': '192.168.1.212:62235',
 'address': '192.168.1.212:62235',
 'node_id': '3ac284d8e86770557b91570f77372cc72de49efdd743b70195bebca7'}

In [22]:
trainer = PPOTrainer(env=IteratedPDEnv, config={ "env": IteratedPDEnv, "env_config": {}})

(RolloutWorker pid=3715394) 2022-03-30 10:54:32,781	ERROR worker.py:430 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=3715394, ip=192.168.1.212, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fb4a4108c40>)
(RolloutWorker pid=3715394)   File "/home/noon/tools/miniconda3/envs/rlib-client/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 462, in __init__
(RolloutWorker pid=3715394)     _validate_env(self.env, env_context=self.env_context)
(RolloutWorker pid=3715394)   File "/home/noon/tools/miniconda3/envs/rlib-client/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1718, in _validate_env
(RolloutWorker pid=3715394)     env.observation_space.dtype != dummy_obs.dtype:
(RolloutWorker pid=3715394) AttributeError: 'NoneType' object has no attribute 'dtype'
(RolloutWorker pid=3715395) 2022-03-30 10:54:32,744	ERROR worker.py:430 -- Exc

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=3715395, ip=192.168.1.212, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f4ded8e9be0>)
  File "/home/noon/tools/miniconda3/envs/rlib-client/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 462, in __init__
    _validate_env(self.env, env_context=self.env_context)
  File "/home/noon/tools/miniconda3/envs/rlib-client/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1718, in _validate_env
    env.observation_space.dtype != dummy_obs.dtype:
AttributeError: 'NoneType' object has no attribute 'dtype'